#### Auto Encoder

This is the auto encoder implementation that is going to be used in our study

In [14]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import BatchNormalization, Dropout
import matplotlib.pyplot as plt
import kagglehub # Gather data from kagglehub

In [15]:
# Loading dataset
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")
file = os.path.join(path, "creditcard.csv")
data = pd.read_csv(file)
 
print(data.shape)

In [ ]:
input_dimension = data.shape[1]
hidden_layer = 16
input_layer = Input(shape=input_dimension,)
Encoder1 = Dense(hidden_layer,activation="tanh")(input_layer)
Encoder2 = Dense(hidden_layer/2,activation="tanh")(Encoder1)
Encoder3 = Dense(hidden_layer/4,activation="tanh")(Encoder2)
Decoder1 = Dense(hidden_layer/4,activation="tanh")(Encoder3)
Decoder2 = Dense(hidden_layer/2,activation="tanh")(Decoder1)
Decoder3 = Dense(input_dimension,activation="tanh")(Decoder2)
AutoEncoderModel = Model(inputs=input_layer,outputs=Decoder3)

